In [1]:
import json
import csv
import os
import pandas as pd
from itertools import combinations, product
data_older = os.path.join(os.path.dirname(globals()['_dh'][0]), 'backup_data_with_detector_results')
print(data_older)
subfolders = [f.path for f in os.scandir(data_older) if f.is_dir()]
model_files = []
for subfolder in subfolders:
    model_files += [f.path for f in os.scandir(subfolder) if f.is_file()]
print(model_files)
# eval_results_folder = os.path.join(os.path.dirname(data_older), 'hhem2/_eval/eval_output/leaderboard_3_hems_with_gooogle_2/')
# file_name = 'leaderboard_summaries_with_predictions.json'
# output_file = 'leaderboard_summaries_with_predictions_selected'

/home/miaoran/FaithBench/backup_data_with_detector_results
['/home/miaoran/FaithBench/backup_data_with_detector_results/Intel/neural-chat-7b-v3-3.csv', '/home/miaoran/FaithBench/backup_data_with_detector_results/01-ai/Yi-1.5-6B-Chat.csv', '/home/miaoran/FaithBench/backup_data_with_detector_results/01-ai/Yi-1.5-9B-Chat.csv', '/home/miaoran/FaithBench/backup_data_with_detector_results/01-ai/Yi-1.5-34B-Chat.csv', '/home/miaoran/FaithBench/backup_data_with_detector_results/openai/GPT-4-Turbo-2024-04-09.csv', '/home/miaoran/FaithBench/backup_data_with_detector_results/openai/gpt-4o.csv', '/home/miaoran/FaithBench/backup_data_with_detector_results/openai/GPT-4o-mini.csv', '/home/miaoran/FaithBench/backup_data_with_detector_results/openai/o1-mini.csv', '/home/miaoran/FaithBench/backup_data_with_detector_results/openai/GPT-3.5-Turbo.csv', '/home/miaoran/FaithBench/backup_data_with_detector_results/openai/GPT-4.csv', '/home/miaoran/FaithBench/backup_data_with_detector_results/openai/o1-preview.

In [2]:
# All criteria
# valid_summary: summary contains at least 5 words
# hhems_disagree: any pair of HHEMv1, HHEM-2.1-Tri, HHEM-2.1-English has disaagreement
# googles_disagree: t5_11b_trueteacher_and_anli != t5_xxl_true_nli_mixture
# hhem2_google_disagree: HHEM-2.1 (triligual or English) and TrueTeacher (t5_11b_trueteacher_and_anli or t5_xxl_true_nli_mixture) has disagreement
# hhem_google_disagree: HHEM(v1 or triligual or English) and TrueTeacher (t5_11b_trueteacher_and_anli or t5_xxl_true_nli_mixture) has disagreement
# any_disagreement: hhems_disagree or googles_disagree or hhem_google_disagree

# criteria = ['valid_summary', 'hhems_disagree', 'googles_disagree', 'hhem_google_disagree']
criteria = ['valid_summary', 'any_disagreement']

In [3]:
# detectors = ['HHEMv1', 'HHEM-2.1', 'HHEM-2.1-English', 'google/t5_11b_trueteacher_and_anli', 'google/t5_xxl_true_nli_mixture']
detectors = ['HHEMv1', 'HHEM-2.1', 'HHEM-2.1-English', 'google/t5_11b_trueteacher_and_anli', 'google/t5_xxl_true_nli_mixture', 'gpt-3.5-turbo', 'gpt-4-turbo', 'gpt-4o']

In [ ]:
def valid_summary(record):
    # print(record['summary'])
    return len(record['summary'].split()) > 4

def hhem_result(score):
    return True if score >= 0.5 else False

def google_result(score):
    return True if score and int(score) == 1 else False

def gpt_result(score):
    return True if score == 1 else False

def hhems_disagree(record):
    '''
    input: a record containing scores from hhem1, hhem21, hhem2free, and google models
    return: boolean
    '''
    all_combs = list(combinations(['HHEMv1', 'HHEM-2.1', 'HHEM-2.1-English'],2))
    for comb in all_combs:
        # if all([record[comb[0]] >= 0.5, 1-record[comb[1]] > 0.5]) or all([record[comb[0]] < 0.5, 1-record[comb[1]] <= 0.5]):
        #     return True
        if hhem_result(record[comb[0]]) != hhem_result(record[comb[1]]):
            return True
    return False
    
def googles_disagree(record):
    return google_result(record['google/t5_11b_trueteacher_and_anli']) != google_result(record['google/t5_xxl_true_nli_mixture'])

def hhem2_google_disagree(record):
    all_combs = list(product(['HHEM-2.1', 'HHEM-2.1-English'], ['google/t5_11b_trueteacher_and_anli', 'google/t5_xxl_true_nli_mixture']))
    for comb in all_combs:
        if (hhem_result(record[comb[0]]) != google_result(record[comb[1]])):
            return True
    return False

def hhem_google_disagree(record):
    all_combs = list(product(['HHEMv1', 'HHEM-2.1', 'HHEM-2.1-English'], ['google/t5_11b_trueteacher_and_anli', 'google/t5_xxl_true_nli_mixture']))
    for comb in all_combs:
        if (hhem_result(record[comb[0]]) != google_result(record[comb[1]])):
            return True
    return False

def gpts_disagreement(record):
    all_combs = list(combinations(['gpt-3.5-turbo', 'gpt-4-turbo', 'gpt-4o'],2))
    for comb in all_combs:
        if gpt_result(record[comb[0]]) != gpt_result(record[comb[1]]):
            return True
    return False

def hhem_gpt_disagreement(record):
    all_combs = list(product(['HHEMv1', 'HHEM-2.1', 'HHEM-2.1-English'], ['gpt-3.5-turbo', 'gpt-4-turbo', 'gpt-4o']))
    for comb in all_combs:
        if (hhem_result(record[comb[0]]) != gpt_result(record[comb[1]])):
            return True
    return False

def google_gpt_disagreement(record):
    all_combs = list(product(['google/t5_11b_trueteacher_and_anli', 'google/t5_xxl_true_nli_mixture'], ['gpt-3.5-turbo', 'gpt-4-turbo', 'gpt-4o']))
    for comb in all_combs:
        if (google_result(record[comb[0]]) != gpt_result(record[comb[1]])):
            return True
    return False

def any_disagreement(record):
    if (hhems_disagree(record) or googles_disagree(record) or hhem_google_disagree(record) or 
        gpts_disagreement(record) or hhem_gpt_disagreement(record) or google_gpt_disagreement(record)):

        return True
# def any_disagreement(record):
#     if (hhems_disagree(record) or googles_disagree(record) or hhem_google_disagree(record)):
#         return True

def check_criteria(criteria, record):
    '''
    Inputs:
        - criteria: list of criteria that needs to be checked
        - record: a record containing scores from hhem1, hhem21, hhem2free, and google models
    Return:
        boolean
    '''
    for c in criteria:
        select_fn = eval(c)
        if not select_fn(record):
            return False
    return True

In [ ]:
selected_models = [
    "openai/GPT-3.5-Turbo",
    "openai/gpt-4o",
    "Qwen/Qwen2.5-7B-Instruct",
    "microsoft/Phi-3-mini-4k-instruct",
    "cohere/command-r-08-2024",
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    "meta-llama/Meta-Llama-3.1-70B-Instruct",
    "google/gemini-1.5-flash-001",
    "Anthropic/claude-3-5-sonnet-20240620",
    "mistralai/Mistral-7B-Instruct-v0.3",
]

In [6]:
summarizer_count = {}
summarizer_count_combination = {}
summarizer_count_each = {}
grouped_by_source = {}
selected_records = []
for file_name in model_files:
    with open(file_name) as f:
        print(file_name)
        data = pd.read_csv(file_name).fillna('')
        
        if selected_models and list(set(data['model'].values.tolist()))[0] not in selected_models:
            continue
        data = data.drop_duplicates(subset=['source', 'summary'], keep='last')
        print(data.shape)
        for record_id, record in data.iterrows():
            # print(record)
            if check_criteria(criteria, record):
                selected_records.append(record)
                detected_halu = [detector for detector in detectors if (not record[detector]) or (record[detector] < 0.5)]
                if record['model'] not in summarizer_count:
                    summarizer_count[record['model']] = 1
                else:
                    summarizer_count[record['model']] += 1

                if record['model'] not in summarizer_count_combination:
                    summarizer_count_combination[record['model']] = {}
                if ' + '.join(detected_halu) not in summarizer_count_combination[record['model']]:
                    summarizer_count_combination[record['model']][' + '.join(detected_halu)] = 1
                else:
                    summarizer_count_combination[record['model']][' + '.join(detected_halu)] += 1

                if record['model'] not in summarizer_count_each:
                    summarizer_count_each[record['model']] = {k:0 for k in ['total'] + detectors}
                summarizer_count_each[record['model']]['total'] += 1
                for detector in detected_halu:
                    summarizer_count_each[record['model']][detector] += 1

summarizer_count = {k: v for k, v in sorted(summarizer_count.items(), key=lambda item: item[1], reverse=True)}
summarizer_count_combination = {k: v for k, v in sorted(summarizer_count_combination.items(), key=lambda item: sum(list((item[1].values()))), reverse=True)}
summarizer_count_each = {k: v for k, v in sorted(summarizer_count_each.items(), key=lambda item: item[1]["total"], reverse=True)}

/home/miaoran/FaithBench/backup_data_with_detector_results/Intel/neural-chat-7b-v3-3.csv
/home/miaoran/FaithBench/backup_data_with_detector_results/01-ai/Yi-1.5-6B-Chat.csv
/home/miaoran/FaithBench/backup_data_with_detector_results/01-ai/Yi-1.5-9B-Chat.csv
/home/miaoran/FaithBench/backup_data_with_detector_results/01-ai/Yi-1.5-34B-Chat.csv
/home/miaoran/FaithBench/backup_data_with_detector_results/openai/GPT-4-Turbo-2024-04-09.csv
/home/miaoran/FaithBench/backup_data_with_detector_results/openai/gpt-4o.csv
(1006, 14)
/home/miaoran/FaithBench/backup_data_with_detector_results/openai/GPT-4o-mini.csv
/home/miaoran/FaithBench/backup_data_with_detector_results/openai/o1-mini.csv
/home/miaoran/FaithBench/backup_data_with_detector_results/openai/GPT-3.5-Turbo.csv
(831, 14)
/home/miaoran/FaithBench/backup_data_with_detector_results/openai/GPT-4.csv
/home/miaoran/FaithBench/backup_data_with_detector_results/openai/o1-preview.csv
/home/miaoran/FaithBench/backup_data_with_detector_results/deepsee

In [7]:
# with open(output_file + '.json', 'w') as f:
#     json.dump(selected_records, f, indent=2)
output_file = '/home/miaoran/FaithBench/backup_data_with_detector_results/hard_examples_selected_models_' + '-'.join(criteria)    
data_file = open(output_file + '.csv', 'w', newline='')
csv_writer = csv.writer(data_file)
count = 0
for data in selected_records:
    if count == 0:
        csv_writer.writerow(['source','summary','model','HHEMv1','HHEM-2.1','HHEM-2.1-English','trueteacher','true_nli', 'gpt-3.5-turbo', 'gpt-4-turbo', 'gpt-4o', 'minicheck-roberta-large', 'minicheck-deberta-v3-large', 'minicheck-flan-t5-large'])
        count += 1
    csv_writer.writerow(data)

print(f'{str(len(selected_records))} selected examples are saved to {output_file}.csv')

1977 selected examples are saved to /home/miaoran/FaithBench/backup_data_with_detector_results/hard_examples_selected_models_valid_summary-any_disagreement.csv


In [8]:

print(json.dumps(dict(sorted(summarizer_count.items())), indent=2))

{
  "Anthropic/claude-3-5-sonnet-20240620": 290,
  "Qwen/Qwen2.5-7B-Instruct": 167,
  "cohere/command-r-08-2024": 168,
  "google/gemini-1.5-flash-001": 208,
  "meta-llama/Meta-Llama-3.1-70B-Instruct": 225,
  "meta-llama/Meta-Llama-3.1-8B-Instruct": 240,
  "microsoft/Phi-3-mini-4k-instruct": 226,
  "mistralai/Mistral-7B-Instruct-v0.3": 235,
  "openai/GPT-3.5-Turbo": 99,
  "openai/gpt-4o": 119
}


In [9]:
print(json.dumps(summarizer_count_combination, indent=2))

{
  "Anthropic/claude-3-5-sonnet-20240620": {
    "HHEM-2.1 + HHEM-2.1-English": 17,
    "HHEM-2.1 + HHEM-2.1-English + google/t5_11b_trueteacher_and_anli": 6,
    "google/t5_11b_trueteacher_and_anli": 88,
    "HHEM-2.1": 30,
    "HHEM-2.1 + google/t5_11b_trueteacher_and_anli": 3,
    "HHEMv1 + google/t5_11b_trueteacher_and_anli + gpt-3.5-turbo": 1,
    "gpt-3.5-turbo": 8,
    "google/t5_11b_trueteacher_and_anli + gpt-3.5-turbo": 7,
    "google/t5_11b_trueteacher_and_anli + google/t5_xxl_true_nli_mixture": 6,
    "google/t5_xxl_true_nli_mixture": 10,
    "HHEMv1": 36,
    "HHEM-2.1 + google/t5_11b_trueteacher_and_anli + google/t5_xxl_true_nli_mixture": 3,
    "HHEMv1 + HHEM-2.1": 3,
    "HHEM-2.1-English": 15,
    "google/t5_11b_trueteacher_and_anli + gpt-4-turbo": 1,
    "gpt-4-turbo": 8,
    "gpt-4o": 1,
    "HHEMv1 + HHEM-2.1-English + google/t5_11b_trueteacher_and_anli": 1,
    "HHEMv1 + HHEM-2.1 + google/t5_11b_trueteacher_and_anli + google/t5_xxl_true_nli_mixture": 4,
    "gpt-4-

In [10]:
print(json.dumps(summarizer_count_each, indent=2))

{
  "Anthropic/claude-3-5-sonnet-20240620": {
    "total": 290,
    "HHEMv1": 60,
    "HHEM-2.1": 87,
    "HHEM-2.1-English": 61,
    "google/t5_11b_trueteacher_and_anli": 144,
    "google/t5_xxl_true_nli_mixture": 34,
    "gpt-3.5-turbo": 27,
    "gpt-4-turbo": 28,
    "gpt-4o": 11
  },
  "meta-llama/Meta-Llama-3.1-8B-Instruct": {
    "total": 240,
    "HHEMv1": 88,
    "HHEM-2.1": 54,
    "HHEM-2.1-English": 38,
    "google/t5_11b_trueteacher_and_anli": 72,
    "google/t5_xxl_true_nli_mixture": 43,
    "gpt-3.5-turbo": 30,
    "gpt-4-turbo": 66,
    "gpt-4o": 44
  },
  "mistralai/Mistral-7B-Instruct-v0.3": {
    "total": 235,
    "HHEMv1": 65,
    "HHEM-2.1": 93,
    "HHEM-2.1-English": 70,
    "google/t5_11b_trueteacher_and_anli": 68,
    "google/t5_xxl_true_nli_mixture": 11,
    "gpt-3.5-turbo": 30,
    "gpt-4-turbo": 76,
    "gpt-4o": 62
  },
  "microsoft/Phi-3-mini-4k-instruct": {
    "total": 226,
    "HHEMv1": 69,
    "HHEM-2.1": 39,
    "HHEM-2.1-English": 42,
    "google/t5_1

In [11]:
from collections import Counter
df = pd.read_csv(f"{output_file}.csv", index_col=False)
print(df.columns)
source_count = dict(Counter(df['source'].values.tolist()))
print(len(source_count))
print(source_count)
# count_counter = dict(Counter(source_count.values()))
# count_counter = sorted(count_counter.items())
# print(count_counter)
candidate_sources = []
for source, count in source_count.items():
    # Initial stage: if count >= 5: # select source doc that occur no less than 5 times in conflict examples
    if count >= 3 and count < 5:
        candidate_sources.append(source)

# GPT-35-Turbo get less records
gpt_35_source = pd.read_csv('/home/miaoran/FaithBench/backup_data_with_detector_results/openai/GPT-3.5-Turbo.csv')['source'].values.tolist()

candidate_sources = set(candidate_sources)
gpt_35_source = set(gpt_35_source)
# print(candidate_sources)
# print(gpt_35_source)
candidate_sources = list(candidate_sources.intersection(gpt_35_source))
print(candidate_sources)
annotation_examples = []
for file_name in model_files:
    with open(file_name) as f:
        data = pd.read_csv(file_name).fillna('')
        
        if selected_models and list(set(data['model'].values.tolist()))[0] not in selected_models:
            continue
        print(file_name)
        data = data.drop_duplicates(subset=['source', 'summary'], keep='last')
        data = data[data['source'].isin(candidate_sources)]
        print(data.shape)
        annotation_examples.append(data)
final_df = pd.concat(annotation_examples)
final_df.to_csv('/home/miaoran/FaithBench/assign/examples_to_annotate2.csv', mode='w', index=False, header=['source','summary','model','HHEMv1','HHEM-2.1','HHEM-2.1-English','trueteacher','true_nli', 'gpt-3.5-turbo', 'gpt-4-turbo', 'gpt-4o', 'minicheck-roberta-large', 'minicheck-deberta-v3-large', 'minicheck-flan-t5-large'])
print(final_df.shape)

Index(['source', 'summary', 'model', 'HHEMv1', 'HHEM-2.1', 'HHEM-2.1-English',
       'trueteacher', 'true_nli', 'gpt-3.5-turbo', 'gpt-4-turbo', 'gpt-4o',
       'minicheck-roberta-large', 'minicheck-deberta-v3-large',
       'minicheck-flan-t5-large'],
      dtype='object')
624
{"Babies given antibiotics in the first six months of life are more likely to be fat as toddlers, a large-scale study has found. The researchers said say that just as antibiotics are used to make farm animals put on weight, the may also be fattening our children. Writing in the respected medical journal Pediatrics, they said that the widely-prescribed drugs could be contributing to the obesity epidemic. A third of 10-11 year olds and more than a fifth of 4-5 year olds in England are overweight or obese, leading to fears that today's generation will be the first to die at an earlier age than their parents. Obesity: Babies given antibiotics in the first six months of life are more likely to be fat as toddlers, a 